In [1]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

### **Crear un DataFrame a partir de un RDD**

In [ ]:
rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, x ** 2)) # Queremos que nos devuelva tuplas.

In [3]:
rdd.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

In [4]:
df = rdd.toDF(['numero', 'cuadrado']) # Le damos una lista con los nombres de columnas que queremos en el DF.

In [5]:
df.printSchema()

root
 |-- numero: long (nullable = true)
 |-- cuadrado: long (nullable = true)



- long es el tipo de dato.

- nullable es si acepta nulos o no.

In [6]:
df.show()

+------+--------+
|numero|cuadrado|
+------+--------+
|     0|       0|
|     1|       1|
|     2|       4|
|     3|       9|
|     4|      16|
|     5|      25|
|     6|      36|
|     7|      49|
|     8|      64|
|     9|      81|
+------+--------+



- Crear un DF a partir de un RDD con schema.

In [7]:
rdd1 = sc.parallelize([(1, 'Jose', 35.5), (2, 'Teresa', 54.3), (3, 'Katia', 12.7)])

- 2 formas para crear el schema.

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

- Vía uno.

In [10]:
esquema1 = StructType(
    [
        StructField('id', IntegerType(), True), # Nombre de la columna, tipo de dato, si admite nulos o no.
        StructField('nombre', StringType(), True),
        StructField('saldo', DoubleType(), True)
    ]
)

Vía dos.

In [11]:
esquema2 = "`id` INT, `nombre` STRING, `saldo` DOUBLE"

In [12]:
df1 = spark.createDataFrame(rdd1, schema = esquema1)

In [13]:
df1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- saldo: double (nullable = true)



In [ ]:
df1.show() # Muestra por default las primeras 20 filas del DF.

+---+------+-----+
| id|nombre|saldo|
+---+------+-----+
|  1|  Jose| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+

